# 10장 여러 데이터 소스를 통합 데이터셋으로 합치기

## 10.1 원본 CT 데이터 파일
CT 데이터는 메타데이터 헤더 정보가 포함된 .mhd 파일과 3차원 배열을 만들 원본 데이터 바이트를 포함하는 .raw파일 두가지로 이루어져 있다. 각 파일 이름은 시리즈 UID로 시리즈 UID 1.2.3의 경우 1.2.3.mhd와 1.2.3.raw 두 가지 파일이 있다. Ct 클래스는 두 파일을 읽어서 3차원 배열을 만들고 환자 좌표계를 배열에서 필요로 하는 인덱스, 행, 열 좌표로 바꿔주는 변환 행렬도 만든다. 

## 10.2 LUNA 애노테이션 데이터 파싱
LUNA에서 제공하는 annotations.csv 파일에서는 좌표 정보, 해당 좌표 지점이 결점인지 여부, CT 스캔에 대한 고유 식별자를 얻을 것으로 기대된다. CSV 파일에서 파싱으로 데이터를 불러올 수 있다.

모든 표준 지도 학습은 훈련셋과 검증셋으로 나뉜다. 여기에서 둘 다 실세계 데이터를 반영해야 하며, 일반적인 방식으로 다루는 것을 가정한다.

![](2022-09-14-15-49-54.png)

![](2022-09-14-15-50-15.png)

배시 셀 세션에서 파일 정보를 볼 수 있다. candidates는 551066개의 행을 가지고 있고, seriesuid, coordX, coordY, coordZ, class라는 헤더들을 가지고 있다. annotations에는 약 1200개의 행이 있고, candidates와는 마지막 헤더 diameter_mm만 다르다. 

### 10.2.1 훈련셋과 검증셋
모든 표준 지도 학습에는 훈련셋과 검증셋으로 나눈다. 둘 다 모두 실세계 데이터를 반영해야 한다. 안타깝게도 annotations.csv에서 제공하는 위치 정보와 candidates.csv의 좌표가 정확하게 일치하지 않을 때도 있다.

![](2022-09-14-15-54-03.png)

### 10.2.2 에노테이션 데이터와 후보 데이터 합치기
이제 앞에 설명한 두 개의 파일을 합치는 getCandidateInfoList 함수를 만들어보자. 각 결절 정보를 담아둘 네임드 튜플을 파일 상단에 두고 사용하자

In [ ]:
from collections import namedtuple
CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xyz',
)

후보 정보 리스트는 순서대로 결절의 상태와 결절의 직경, 올바른 CT 스캔에 배치하기 위한 순변과 큰 CT에서 후보를 찾기 위한 중심점을 갖는다. 

일부 데이터 파일은 파싱에 시간이 걸리므로 함수 호출 결과를 메모리에 캐시한다. 

![](2022-09-14-15-59-50.png)

requireOnDisk_bool 파라미터를 사용하여 디스크상에서 시리즈 UID가 발견되는 LUNA 데이터만 사용하고 이에 해당하는 엔트리만 CSV 파일에서 걸러 사용할 예정이다. 

In [ ]:
def getCandidateInfoList(requireOnDisk_bool=True):
        mhd_list = glob.glob('data-unversioned/part2/luna/subset*/*.mhd') # LUNA 데이터를 받은 주소를 통해 이름이 subset#.mhd인 파일을 리스트로 반환하는 glob함수
        presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list} # 디스크에 있는 데이터를 찾기 위한 set

후보 정보를 얻었다면 annotations.csv의 직경 정보를 합치자. 에노테이션 정보는 series_uid로 그룹화하여 두 파일에서 일치하는 행을 찾는 키로 사용하자. 



In [ ]:
diameter_dict = {}
with open('data/part2/luna/annotations.csv', "r") as f:#annotations.csv 파일 불러오기
        for row in list(csv.reader(f))[1:]: #첫 번째 행(칼럼 헤더)를 제외하고 전부 호출
            series_uid = row[0] #series_uid를 키로 사용
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]]) # csv 파일에서 center_xyz
            annotationDiameter_mm = float(row[4]) #결절의 직경 길이

            diameter_dict.setdefault(series_uid, []).append(
                (annotationCenter_xyz, annotationDiameter_mm)
            )

이제 candidates.csv의 정보를 사용하여 전체 후보 리스트를 만들자



In [ ]:
candidateInfo_list = []
with open('data/part2/luna/candidates.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]

            if series_uid not in presentOnDisk_set and requireOnDisk_bool: # series_uid가 없으면 서브셋에는 있지만 디스크에는 없으므로 건너뜀
                continue

            isNodule_bool = bool(int(row[4])) # 결절인지 여부
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            candidateDiameter_mm = 0.0
            for annotation_tup in diameter_dict.get(series_uid, []):
                annotationCenter_xyz, annotationDiameter_mm = annotation_tup
                for i in range(3):
                    delta_mm = abs(candidateCenter_xyz[i] - annotationCenter_xyz[i]) # 직경
                    if delta_mm > annotationDiameter_mm / 4: # 반경을 얻기 위해 직경을 2로 나누고, 두 개의 결절 센터가 결절의 크기 기준으로 
                                                             #너무 떨어져 있는지를 반지름의 절반 길이를 기준으로 판점
                        break
                else:
                    candidateDiameter_mm = annotationDiameter_mm
                    break

            candidateInfo_list.append(CandidateInfoTuple(
                isNodule_bool,
                candidateDiameter_mm,
                series_uid,
                candidateCenter_xyz,
            )) # 전체 후보 리스트
candidateInfo_list.sort(reverse=True)#모든 결절 샘플 내림차순 정렬, 그 뒤에는 결절이 아닌 샘플이 이어져 있는 데이터 준비됨
return candidateInfo_list

### 10.3 개별 CT 스캔 로딩
다음은 디스크에서 CT 데이터를 얻어와 파이썬 객체로 변환해서 3차원 결절 밀도 데이터로 사용할 수 있도록 만드는 작업이다.

![](2022-09-14-16-27-56.png)

 SimpleITK를 사용하여 MetaIO 포맷으로 되어있는 데이터를 친숙한 넘파이 배열로 읽어들일 수 있다.

In [ ]:
class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob(
            'data-unversioned/part2/luna/subset*/{}.mhd'.format(series_uid) #series_uid를 format 함수로 mhd 파일 불러오기
        )[0]

        ct_mhd = sitk.ReadImage(mhd_path) #SimpleITK.ReadImage로 mhd 파일 뿐만아니라 .raw 파일도 읽는다.
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32) # 값의 타입을 np.float32로 변환하기 위해 np.array를 다시 만듬

### 10.3.1 하운스필드 단위
우리는 데이터가 저장되는 비트보다는 데이터 자체를 이해해야 한다. CT 스캔 복셀은 하운스필드 단위(HU)로 표시한다. -1000이하일 경우 공기이므로 버리고 뼈가 1000HU이므로 1000 이상일 시에도 clip함수로 잘라내자. 데이터에서 이와 같은 이상값을 제거하는 과정도 중요하다. 이제 이렇게 만들어진 값을 self에 할당한다.

In [ ]:
        ct_a.clip(-1000, 1000, ct_a)
        self.series_uid = series_uid
        self.hu_a = ct_a

## 10.4 환자 좌표계를 사용해 결절 위치 정하기

딥러닝 모델은 고정된 크기의 입력을 필요로 한다. 모델의 훈련에는 CT 스캔에서 깔끔하게 잘라낸 중심이 잘 잡힌 후보 데이터를 사용해서 상대적으로 모델이 수행할 작업을 수월하게 만들자

### 10.4.1 환자 좌표계
10.2절에서 읽어들인 후보 중심점 데이터는 복셀이 아니라 밀리미터 단위로 표시되어 있다. 현재 표현된 밀리미터 기반 좌표계인 (X, Y, Z)를 복셀 주소 기반 좌표계인 (I, R, C)로 변환해야 한다.

![](2022-09-14-16-41-54.png)

환자 좌표계는 위치 기준을 임의로 잡기 때문에 CT 복셀 배열의 기준과는 일치하지 않는다.

![](2022-09-14-16-43-56.png)

### 10.4.2 CT 스캔 형태와 복셀 크기
복셀은 정육면체가 아니다, 1.125mmx1.125mmx2.5mm크기 또는 유사한 크기를 가진다. 따라서 왜곡된 이미지를 보이게 하므로, 실제 비율로 보려면 비율 계수를 적용해야 한다. 

![](2022-09-14-16-47-24.png)

### 10.4.3 밀리미터를 복셀 주소로 변환하기
밀리미터 좌표(_xyz로 끝남)을 IRC 배열 좌표(_irc로 끝남)으로 변환하기 위해 유틸리티 코드를 정의하자. 축을 뒤집는 것(회전 등의 변환)은 ct_mhd.GetDirections()가 반환하는 튜플에 3x3 행렬로 인코딩되어 있다. 다음 네 단계를 순서대로 따라서 복셀 인덱스를 좌표로 바꿔보자
1. 좌표를 XYZ 체계로 만들기 위해 IRC에서 CRI로 뒤집는다
2. 인덱스를 복셀 크기로 확대축소한다
3. 파이썬의 @를 사용하여 방향을 나타내는 행렬과 행렬곱을 수행한다
4. 기준으로부터 오프셋을 더한다.

In [ ]:
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])

def irc2xyz(coord_irc, origin_xyz, vxSize_xyz, direction_a):
    cri_a = np.array(coord_irc)[::-1] # IRC를 CRI로 뒤집음
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz) #인덱스를 복셀 크기로 확대축소해주는 array
    coords_xyz = (direction_a @ (cri_a * vxSize_a)) + origin_a # 확대 축소 된 cri_a를 방향을 나타내는 행렬과 행렬곱을 수행한 후 기준으로부터 오프셋 더함
    return XyzTuple(*coords_xyz)
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))

XYZ를 IRC로 변환할 때 필요한 메타데이터는 CT 데이터와 함께 들어 있는 MetaIO 파일에 있다. .mhd 파일에서 복셀 크기와 포지셔닝 메타데이터를 꺼낼 수 있다.

### 10.4.4 CT 스캔에서 결절 추출하기
앞에서 설명했다 싶이 CT 스캔의 99.9999%는 실제 결절 부분이 아니다. 떄문에 우리는 각 후보 영역을 추출해 모델이 한번에 한 영역에 집중 할 수 있도록 만들 예정이다.

![](2022-09-14-17-01-13.png)

getRawNodule() 함수는 CSV 데이터에 명시된 (XYZ)로 표시된 중심 정보와 복셀 단위의 너비 정보도 입력 받아 CT 덩어리와 배열 좌표로 변환된 후보의 중심값을 반환한다.

In [ ]:
def getRawCandidate(self, center_xyz, width_irc):
        center_irc = xyz2irc(
            center_xyz,
            self.origin_xyz,
            self.vxSize_xyz,
            self.direction_a,
        )

        slice_list = []
        for axis, center_val in enumerate(center_irc):
            start_ndx = int(round(center_val - width_irc[axis]/2))
            end_ndx = int(start_ndx + width_irc[axis])
            slice_list.append(slice(start_ndx, end_ndx))

        ct_chunk = self.hu_a[tuple(slice_list)]

        return ct_chunk, center_irc

## 10.5 간단한 데이터셋 구현
이번에는 Datset 인스턴스를 직접 구현해보자. 일단 Dataset 서브클래싱에 필요한 요구사항으로 싲가해서 거꾸로 가면서 작업해보자. 커스텀 서브클래스는 파이토치 API가 요구하는 두 함수만 구현하면 된다
- 초기화 후에 하나의 상수값을 반환해야 하는 __len__구현
- 인덱스를 인자로 받아 훈련에서 사용할 샘플 데이터 튜플을 반환하는 __getitem__메소드
__len__함수는 구현이 어렵지 않다. 우리가 가지고 있는 후보 리스트 하나하나는 샘플이며 이 샘플의 개수를 반환하면 된다.

In [ ]:
def __len__(self):
    return len(self.candidateInfo_list)

__getitem__함수는 이보다는 복잡한데, ndx인자를 받아 0~ndx-1까지의 네 개의 아이템이 있는 샘플 튜플을 반환한다. 이를 위해 self.candidateInfo_list를 구현하고 getCtRawNodule 함수를 제공해야 한다.

In [ ]:
def __getitem__(self, ndx):
        candidateInfo_tup = self.candidateInfo_list[ndx]
        width_irc = (32, 48, 48)#candidate_a의 차원 정보(깊이,높이,너비)

        candidate_a, center_irc = getCtRawCandidate(
            candidateInfo_tup.series_uid,
            candidateInfo_tup.center_xyz,
            width_irc,
        )

이후에는 사용할 데이터를 적절한 타입과 배열 차원으로 준비해야 한다

In [ ]:
candidate_t = torch.from_numpy(candidate_a)#넘파이를 텐서로 변환
candidate_t = candidate_t.to(torch.float32)#float32 실수형으로 변환
candidate_t = candidate_t.unsqueeze(0)#채널 차원 추가

마지막으로 분류 텐서를 만들어야 한다. 여기서 두 가지 요소 중 하나는 가능한 후보 클래스(결절인지 아닌지, 양성/음성인지)이다. 

In [ ]:
pos_t = torch.tensor([
                not candidateInfo_tup.isNodule_bool,
                candidateInfo_tup.isNodule_bool
            ],
            dtype=torch.long,
        )

최종 샘플 튜플을 확인하자